### Yunjo Lee et Jean Marie Vilaire


In [1]:
using LinearAlgebra
using Pkg

In [2]:
Pkg.add("BenchmarkTools")

   Resolving package versions...
  No Changes to `C:\Users\jevilf\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\jevilf\.julia\environments\v1.10\Manifest.toml`


In [3]:
function backsolve(R::UpperTriangular, b)
    x = similar(b)
    n = length(b)
    
    for i in n:-1:1
        if R[i, i] == 0
            throw(ArgumentError("Matrice singuliere a la ligne $i."))
        end
        x[i] = (b[i] - dot(R[i, i+1:n], x[i+1:n])) / R[i, i]
    end
    return x
end

#--------------------------------------------------------------------
function forwardsolve(L::LowerTriangular, b)
    x = similar(b)
    n = length(b)
    
    for i in 1:n
        if L[i, i] == 0
            throw(ArgumentError("Matrice singuliere a la ligne $i."))
        end
        x[i] = (b[i] - dot(L[i, 1:i-1], x[1:i-1])) / L[i, i]
    end
    return x
end

forwardsolve (generic function with 1 method)

In [86]:
# Fonction pour permuter deux lignes et colonnes d'une matrice
function perm_lc(A::Matrix, P::Matrix, L::Matrix, k::Int, pivot_idx::Int)
    if pivot_idx != k
        P[[k, pivot_idx], :] = P[[pivot_idx, k], :]
        A[[k, pivot_idx], :] = A[[pivot_idx, k], :]
        A[:, [k, pivot_idx]] = A[:, [pivot_idx, k]]
        L[[k, pivot_idx], 1:k-1] = L[[pivot_idx, k], 1:k-1]
    end
end


perm_lc (generic function with 1 method)

In [94]:
using LinearAlgebra

function bk_patial_pivot(A)
    n = size(A, 1)
    α = (1 + sqrt(17)) / 8

    for k in 1:n
        if k < n
            λ = maximum(abs.(A[k+1:end, k]))
            if λ > 0
                if abs(A[k, k]) >= α * λ
                    println("Utiliser A[$k, $k] comme pivot 1-par-1")
                else
                    r = argmax(abs.(A[k+1:end, k]))[1] + k
                    σ = maximum(abs.(A[:, r]))
                    if abs(A[k, k]) * σ >= α^2 * λ^2
                        println("Utiliser A[$k, $k] comme pivot 1-par-1")
                    elseif abs(A[r, r]) >= α * σ
                        println("Utiliser A[$r, $r] comme pivot 1-par-1")
                    
                    else
                        println("Utiliser A[$k, $k] et A[$r, $r] comme pivot 2-par-2")
                    end
                end
            end
        else
            if abs(A[k, k]) > 0
                println("Utiliser A[$k, $k] comme pivot 1-par-1")
            end
        end
    end
end

# Exemple d'utilisation
A = rand(10,10)
bk_patial_pivot(A)


Utiliser A[1, 1] comme pivot 1-par-1
Utiliser A[2, 2] comme pivot 1-par-1
Utiliser A[8, 8] comme pivot 1-par-1
Utiliser A[4, 4] comme pivot 1-par-1
Utiliser A[8, 8] comme pivot 1-par-1
Utiliser A[8, 8] comme pivot 1-par-1
Utiliser A[8, 8] comme pivot 1-par-1
Utiliser A[8, 8] comme pivot 1-par-1
Utiliser A[9, 9] comme pivot 1-par-1
Utiliser A[10, 10] comme pivot 1-par-1


In [92]:
function bp_complete_pivot(A)
    n = size(A, 1)
    α = (1 + sqrt(17)) / 8
    pivots = []

    for k in 1:n-1
        ξ = maximum(abs.(A[i, j]) for i in 1:n, j in 1:n if i != j)
        η = maximum(abs.(A[i, i]) for i in 1:n)
        if ξ > 0 || η > 0
            λ = maximum(abs.(A[k, :]))
            if abs(A[k, k]) >= α * λ
                if η >= ξ
                    println("Utiliser A[$k, $k] comme pivot 1-par-1")
                    push!(pivots, (k, k, 1))
                else
                    println("Utiliser A[$k, $k] et A[$(k+1), $(k+1)] comme pivot 2-par-2")
                    push!(pivots, (k, k+1, 2))
                end
            end
        end
    end

    return pivots
end

A = [1.0  1.0  1.0   1.0   1.0;
     1.0  3.0  3.0   3.0   3.0;
     1.0  3.0  6.0   7.0   7.0;
     1.0  3.0  7.0  12.0  12.0;
     1.0  3.0  7.0  12.0  17.0]

pivots = bp_complete_pivot(copy(A))
println("Pivots = ", pivots)


Utiliser A[1, 1] comme pivot 1-par-1
Utiliser A[2, 2] comme pivot 1-par-1
Utiliser A[3, 3] comme pivot 1-par-1
Utiliser A[4, 4] comme pivot 1-par-1
Pivots = Any[(1, 1, 1), (2, 2, 1), (3, 3, 1), (4, 4, 1)]


In [79]:
S = bunchkaufman(A)
S.D

5×5 Tridiagonal{Float64, Vector{Float64}}:
 0.666667  0.0       ⋅        ⋅         ⋅ 
 0.0       1.43478  0.0       ⋅         ⋅ 
  ⋅        0.0      1.91667  0.0        ⋅ 
  ⋅         ⋅       0.0      3.52941   0.0
  ⋅         ⋅        ⋅       0.0      17.0

In [85]:
using LinearAlgebra

function bk(A::AbstractMatrix)
    n = size(A, 1)
    α = (1 + sqrt(17)) / 8
    L = Matrix{Float64}(I, n, n)
    D = copy(A)
    P = Matrix{Float64}(I, n, n)

    for i in 1:n-1
        λ = maximum(abs.(A[i, :]))
        if any(D[i:end, i:end] .> α * λ)
            val, p = findmax(abs.(D[i:end, i]))
            ind = p + i - 1
            perm_lc(D, P, L, i, ind)

            L[i+1:end, i] .= D[i+1:end, i] / D[i, i]
            D[i+1:end, i] .= 0.0
            D[i+1:end, i+1:end] .-= L[i+1:end, i] * D[i, i+1:end]'
            D[i, i+1:end] .= 0.0
        else
            break
        end
    end

    return L, Diagonal(D), P
end

A = [4.0 1.0 2.0 3.0;
     1.0 2.0 0.0 1.0;
     2.0 0.0 3.0 2.0;
     3.0 1.0 2.0 4.0]

L, D, P = bk(A)
display(L')
display(D)
display(P)


4×4 adjoint(::Matrix{Float64}) with eltype Float64:
 1.0  0.25   0.5       0.75
 0.0  1.0   -0.285714  0.142857
 0.0  0.0    1.0       0.0
 0.0  0.0    0.0       1.0

4×4 Diagonal{Float64, Vector{Float64}}:
 4.0   ⋅     ⋅        ⋅ 
  ⋅   1.75   ⋅        ⋅ 
  ⋅    ⋅    1.85714   ⋅ 
  ⋅    ⋅     ⋅       1.71429

4×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0